In [1]:
import keras
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, Dropout, BatchNormalization, Dense, Flatten, LayerNormalization
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.applications as appl
from sklearn.model_selection import train_test_split
import os
import cv2
import time
import random
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.metrics as metrics
from tqdm import tqdm
from tensorflow.keras.utils import Sequence
from keras import callbacks  
from keras.models import load_model
from keras.utils import np_utils

In [2]:
load_path='F:/VA/onehandtwohand/128/106words_DSLR_FH/'

model_name1 = '15layer_lr0.00001_106words_dslr128-99.79'

CATEGORIES=np.load(load_path+'cat_106.npy', allow_pickle=True)
IMG_SIZE=128
cat_len=len(CATEGORIES)
print(cat_len)

106


In [3]:
# #load saved history
history_const=np.load(load_path+model_name1+'_history.npy',allow_pickle='TRUE').item()

# #load saved model
model1=load_model(load_path+model_name1+'_model.h5')

model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 32)      2432      
                                                                 
 batch_normalization (BatchN  (None, 128, 128, 32)     128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      25632     
                                                                 
 conv2d_2 (Conv2D)           (None, 128, 128, 64)      51264     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 64, 64)       0         
 )                                                               
                                                                 
 batch_normalization_1 (Batc  (None, 64, 64, 64)       2

In [4]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results
    
def draw_landmarks(image, results):   
    #face
    mp_drawing.draw_landmarks(
        image,
        results.face_landmarks,
        mp_holistic.FACEMESH_CONTOURS,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp_drawing_styles
        .get_default_face_mesh_contours_style())
#     #pose
#     mp_drawing.draw_landmarks(
#         image,
#         results.pose_landmarks,
#         mp_holistic.POSE_CONNECTIONS,
#         landmark_drawing_spec=mp_drawing_styles
#         .get_default_pose_landmarks_style())
    
    #left hand
    mp_drawing.draw_landmarks(
        image,
        results.left_hand_landmarks,
        mp_holistic.HAND_CONNECTIONS,
#         landmark_drawing_spec=None,
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style())
    # right hand
    mp_drawing.draw_landmarks(
        image,
        results.right_hand_landmarks,
        mp_holistic.HAND_CONNECTIONS,
#         landmark_drawing_spec=None,
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style())

In [5]:
def unique1(x):
    return np.unique(x)

In [6]:
## For veryyyyyyyy beautiful webcam input:
import statistics

cap = cv2.VideoCapture(0)
org = (20, 20)  
org1 = (310, 20) 
fontScale = 0.65  
font = cv2.FONT_HERSHEY_SIMPLEX
# Blue color in BGR
color = (130, 0, 0)  
# Line thickness of 2 px
thickness = 1 
thickness1 = -1
start_point = (0,0)
end_point = (480,30)
color1 = (255, 255, 255)  

capture_duration = 8

with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    freq_arr = []
    start_time = time.time()
    while( int(time.time() - start_time) < capture_duration ):
        
        #image from feeder
        r, img_array = cap.read()
        
        img_array = cv2.flip(img_array, 1)
        
        #webcam
        img_array = img_array[:, 224:800, :]
#         #dslr
#         img_array = cv2.resize(img_array[:, 224:800, :],(480,480))
        
        image, results = mediapipe_detection(img_array, holistic)
        draw_landmarks(image, results)
        if not (results.left_hand_landmarks or results.right_hand_landmarks):
            continue

        # white background
        img = np.zeros([480,480,3],dtype=np.uint8)
        img.fill(255) 
        draw_landmarks(img, results)

        # for prediction
        IMG_SIZE=128
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        X = np.array(img).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
        X = X.astype('float32')
        X /= 255
        X = np.array(X)
        Y = model1.predict(X,verbose=0)

        image1 = cv2.rectangle(img_array, start_point, end_point, color1, thickness1)
        image1 = cv2.rectangle(image1, (0,30), (480,30), color, 2)
        image1 = cv2.putText(image1,"Prediction: "+ CATEGORIES[np.argmax(Y)], org, font, 
               fontScale, color, thickness, cv2.LINE_AA)
        image1 = cv2.putText(image1,"Left: "+  str(int(capture_duration-(time.time() - start_time))), org1, font, 
               fontScale, color, thickness, cv2.LINE_AA)
#         image1 = cv2.putText(image1,"Accuracy:"+ str(int("%.2f"% np.max(Y))), org1, font, 
#                fontScale, color, thickness, cv2.LINE_AA)
        
        image1 = cv2.resize(image1, (750, 750))
        freq_arr.append([CATEGORIES[np.argmax(Y)],time.time()])
        
        cv2.imshow('Realtime testing', image1)
        if cv2.waitKey(10) & 0xFF == ord('q'):
                break

# close the camera
cap.release()
# writer.release()
# close all the opened windows
cv2.destroyAllWindows()

#sample words:
dupatta, saree, goat, dance,monkey, lope, handkerchief, school, rain, peacock, sun, 
idea, black, hair, chicken, owl, cash, frog


In [7]:
from collections import Counter

freq_np =np.array(freq_arr)
result = [item for items, c in Counter(freq_np[:,0]).most_common()
                                      for item in [items] * c]
u, idx = np.unique(result, return_index=True)
sort=u[np.argsort(idx)]
iddr=[]
for i in range(2):
    iddr.append(np.where(freq_np == sort[i])[0][0])

if iddr[0]>iddr[1]:
    sen=sort[1]+' '+sort[0]+'.'
else:
    sen=sort[0]+' '+sort[1]+'.'
print(sen.capitalize())    
    

D owl.


In [9]:
# print( freq_np[:,0].index('cow'))
print(freq_arr)

[['D', 1681887795.4473903], ['frog', 1681887795.5783925], ['owl', 1681887795.7351997], ['owl', 1681887795.8766587], ['owl', 1681887796.0176933], ['owl', 1681887796.1586938], ['cat', 1681887796.3002326], ['cat', 1681887796.4568908], ['cat', 1681887796.614055], ['cat', 1681887796.755135], ['owl', 1681887796.9122798], ['owl', 1681887797.0533228], ['owl', 1681887797.2104366], ['cow', 1681887797.372596], ['D', 1681887797.5241764], ['E', 1681887797.673283], ['D', 1681887797.806773], ['D', 1681887797.9478464], ['bottle', 1681887798.1049814], ['bottle', 1681887798.2617223], ['D', 1681887798.418851], ['E', 1681887798.5755365], ['Q', 1681887798.7166092]]
